# Data Librarian - Modul 2 PyTerrier Tutorial
## This notebook is based on a CIKM workshop
https://github.com/terrier-org/cikm2021tutorial/blob/main/notebooks

In [1]:
#learn more about pyterrier: https://pyterrier.readthedocs.io/en/latest/

In [1]:
# install pyterrier
# %pip install python-terrier
# %pip install nltk^

In [3]:
import pyterrier as pt
import requests
import pandas as pd
import pickle
import nltk
import numpy as np
import os

## Set `JAVA_HOME` var

In [26]:
os.environ["JAVA_HOME"] = "./jre/Contents/Home/"

## Load data

In [27]:
query = "artificial intelligence"
url =  f"https://www.bibsonomy.org/json/search/{query}?items=1000"

In [28]:
res = requests.get(url)

In [29]:
data = res.json()

In [30]:
df_ai = pd.DataFrame(data["items"])
df_ai = df_ai[df_ai["type"] == "Publication"]

In [31]:
df_ai.head()

,type,id,tags,intraHash,label,user,description,date,changeDate,count,...,keyword,issue,pdf,broken,optseries,position,category,impact,quartile,citedreferences
1000,Publication,https://www.bibsonomy.org/bibtex/24e338b04bc2a...,[710-714],4e338b04bc2abafd9fa0a1f4fe79103a,Artificial Intelligence in Agriculture,ijtsrd,,2021-04-13 13:15:04,2021-04-13 13:15:04,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001,Publication,https://www.bibsonomy.org/bibtex/29f3b59303571...,"[Artificial, Intelligence, Risks, and, Benefits]",9f3b593035714a894c265f4fb9c535e7,Artificial Intelligence Benefit and Risks,ijtsrd,,2020-05-14 10:40:49,2020-05-14 10:40:49,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002,Publication,https://www.bibsonomy.org/bibtex/24bc08fb79365...,"[ANN, Fuzzylogic, ElectricalEngineering, Artif...",4bc08fb79365b5bc698af1cd97fc1a75,Artificial Intelligence in Power Station,ijtsrd,,2020-01-10 09:23:37,2020-01-10 09:23:37,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1003,Publication,https://www.bibsonomy.org/bibtex/29f87881e4dec...,"[Artificial, Intelligence, Markup, Chatbot, La...",9f87881e4decfde15a1af630c8ea27d5,Artificial Intelligence Based Training and Pla...,ijtsrd,,2019-03-26 12:35:44,2019-03-26 12:35:44,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1004,Publication,https://www.bibsonomy.org/bibtex/263320f0ab96a...,"[Learning,, Systems,, Artificialintelligence, ...",63320f0ab96ae778b51b2376aa914f59,The Significance of Artificial Intelligence in...,ijtsrd,,2023-10-14 13:59:49,2023-10-14 13:59:49,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Clean data

In [32]:
#drops columns where at least 50% of the rows have missing values
df_ai_filtered = df_ai.dropna(axis=1, thresh=len(df_ai)*0.5)

In [33]:
df_ai_filtered.head(10)

,type,id,tags,intraHash,label,user,description,date,changeDate,count,url,interHash,pub-type,year,author,authors,volume,pages,bibtexKey,publisher
1000,Publication,https://www.bibsonomy.org/bibtex/24e338b04bc2a...,[710-714],4e338b04bc2abafd9fa0a1f4fe79103a,Artificial Intelligence in Agriculture,ijtsrd,,2021-04-13 13:15:04,2021-04-13 13:15:04,1,https://www.ijtsrd.com/engineering/electrical-...,4d2f7caf67c19f0ff98f5cb8a04fb14d,article,2021,[Matthew N. O. Sadiku | Sarhan M. Musa | Abayo...,[{'first': 'Matthew N. O. Sadiku | Sarhan M. M...,5,710-714,conf/icegov/MalhotraA20,NaN
1001,Publication,https://www.bibsonomy.org/bibtex/29f3b59303571...,"[Artificial, Intelligence, Risks, and, Benefits]",9f3b593035714a894c265f4fb9c535e7,Artificial Intelligence Benefit and Risks,ijtsrd,,2020-05-14 10:40:49,2020-05-14 10:40:49,1,https://www.ijtsrd.com/computer-science/artifi...,ebd51221b66e0a22976b6bf55dcc8b89,article,2020,[Seeta M. Chauhan],"[{'first': 'Seeta M.', 'last': 'Chauhan'}]",4,1007-1009,noauthororeditor,NaN
1002,Publication,https://www.bibsonomy.org/bibtex/24bc08fb79365...,"[ANN, Fuzzylogic, ElectricalEngineering, Artif...",4bc08fb79365b5bc698af1cd97fc1a75,Artificial Intelligence in Power Station,ijtsrd,,2020-01-10 09:23:37,2020-01-10 09:23:37,1,https://www.ijtsrd.com/engineering/electrical-...,2b5bf53f5cc143129b642869065d06cc,article,2019,[P. Naveen | S. Nikitha | P. Sudeesh | V. Vais...,[{'first': 'P. Naveen | S. Nikitha | P. Sudees...,4,1020-1025,noauthororeditor,NaN
1003,Publication,https://www.bibsonomy.org/bibtex/29f87881e4dec...,"[Artificial, Intelligence, Markup, Chatbot, La...",9f87881e4decfde15a1af630c8ea27d5,Artificial Intelligence Based Training and Pla...,ijtsrd,,2019-03-26 12:35:44,2019-03-26 12:35:44,1,http://www.ijtsrd.com/computer-science/artific...,4fd6139b2f85d67ccf1a1943139555f0,article,2018,[Krishanu Deb | Pankaj Agrawal | Harish Nawale...,[{'first': 'Krishanu Deb | Pankaj Agrawal | Ha...,3,1057-1060,noauthororeditor,NaN
1004,Publication,https://www.bibsonomy.org/bibtex/263320f0ab96a...,"[Learning,, Systems,, Artificialintelligence, ...",63320f0ab96ae778b51b2376aa914f59,The Significance of Artificial Intelligence in...,ijtsrd,,2023-10-14 13:59:49,2023-10-14 13:59:49,1,https://www.ijtsrd.com/computer-science/artifi...,ce9618fb026d6db7108c7254d417554a,article,2023,[Dr. Atul Kumar Mishra],"[{'first': 'Dr. Atul Kumar', 'last': 'Mishra'}]",7,786-788,noauthororeditor,NaN
1005,Publication,https://www.bibsonomy.org/bibtex/2f76cb3964bef...,[explanation],f76cb3964befb565e8456efb255b9918,Explanation in artificial intelligence: Insigh...,brusilovsky,Explanation in artificial intelligence: Insigh...,2023-06-05 00:07:52,2023-06-05 00:07:52,2,https://www.sciencedirect.com/science/article/...,924bf61b21094d9c52e30f705ff72365,article,2019,[Tim Miller],"[{'first': 'Tim', 'last': 'Miller'}]",267,1-38,MILLER20191,NaN
1006,Publication,https://www.bibsonomy.org/bibtex/219d04dda236d...,"[simulation, ai]",19d04dda236da5a4e7592a9eb2d4f5eb,A framework for Integrating Artificial Intelli...,mardukasoka,A framework for Integrating Artificial Intelli...,2017-04-29 16:53:32,2017-04-29 16:53:32,2,http://dx.doi.org/10.1007/BF00851350,7f175ce43cdc3c37efa038de41588165,article,1994,"[Georgios I. Doukidis, Marios C. Angelides]","[{'first': 'Georgios I.', 'last': 'Doukidis'},...",8,55--85,Doukidis1994,NaN
1007,Publication,https://www.bibsonomy.org/bibtex/28a86ca74e262...,"[(AI),, Behavior,, Intelligent, Artificialinte...",8a86ca74e262b9883c8dd4a7c60f1652,Artificial Intelligence Role in Modern Science...,ijtsrd,,2024-01-05 10:08:27,2024-01-05 10:08:27,1,https://www.ijtsrd.com/computer-science/artifi...,1c66e4c94b299521b1f28370ecc7d992,article,2023,[Manish Verma],"[{'first': 'Manish', 'last': 'Verma'}]",7,335-342,journals/orl/LeiAG93,NaN
1008,Publication,https://www.bibsonomy.org/bibtex/2b31f13f50e07...,"[Human, Intelligence,, Creativity,, Artificial...",b31f13f50e07ac7e22de4ea993ba89ee,Artificial Intelligence Catalyzes a Revolution...,ijtsrd,,2022-07-22

In [34]:
df_ai_filtered.columns

Index(['type', 'id', 'tags', 'intraHash', 'label', 'user', 'description',
       'date', 'changeDate', 'count', 'url', 'interHash', 'pub-type', 'year',
       'author', 'authors', 'volume', 'pages', 'bibtexKey', 'publisher'],
      dtype='object')

In [35]:
#lets only use columns which seems useful for us
useful_coulmns = ['id', 'tags', 'label', 'description', 'date', 'changeDate', 'url', 'pub-type', 'year', 'author', 'authors', 'publisher']
df_ai_filtered = df_ai_filtered[useful_coulmns]

## PyTerrier init

In [36]:
#initialise pyterrier
if not pt.started():
    pt.init()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [37]:
#pyterrier exspects a docno field
df_ai_filtered['docno'] = df_ai_filtered['id']
df_ai_filtered['text'] = df_ai_filtered['label']

In [ ]:
#excursus indexing

### Create `pt.DFIndexer`

In [38]:
#now we start to index our data
#more details at https://pyterrier.readthedocs.io/en/latest/terrier-indexing.html

index_folder = "./ai_index"

indexer = pt.DFIndexer(index_folder, overwrite=True)
index_ref = indexer.index(df_ai_filtered['text'], df_ai_filtered['docno'])

print(f"path to our index: {index_ref.toString()}")

path to our index: ./ai_index/data.properties


In [39]:
#load our index
index = pt.IndexFactory.of(index_ref)

### Get Stats

In [40]:
print(index.getCollectionStatistics().toString())

Number of documents: 1000
Number of terms: 1379
Number of postings: 6113
Number of fields: 0
Number of tokens: 6351
Field names: []
Positions:   false



In [41]:
#term -> term_id Nt Tf
#Nt: In how many documents does the term occur / frequency
#TF: How often does the term occur in total

for kv in index.getLexicon():
    print(f"{kv.getKey()} -> {kv.getValue().toString()}")

0 -> term821 Nt=1 TF=1 maxTF=1 @{0 0 0}
02 -> term588 Nt=2 TF=2 maxTF=1 @{0 2 2}
1 -> term327 Nt=2 TF=2 maxTF=1 @{0 5 6}
10 -> term542 Nt=3 TF=3 maxTF=1 @{0 9 6}
10th -> term580 Nt=2 TF=2 maxTF=1 @{0 15 4}
11 -> term118 Nt=3 TF=3 maxTF=1 @{0 19 6}
11th -> term1135 Nt=4 TF=4 maxTF=1 @{0 25 4}
12 -> term381 Nt=2 TF=2 maxTF=1 @{0 32 4}
12th -> term1090 Nt=4 TF=4 maxTF=1 @{0 36 0}
13 -> term117 Nt=5 TF=5 maxTF=1 @{0 43 2}
13th -> term1206 Nt=3 TF=3 maxTF=1 @{0 52 4}
14 -> term525 Nt=4 TF=4 maxTF=1 @{0 57 6}
14th -> term1045 Nt=4 TF=4 maxTF=1 @{0 65 0}
15 -> term204 Nt=4 TF=4 maxTF=1 @{0 73 2}
15th -> term1055 Nt=4 TF=4 maxTF=1 @{0 78 4}
16 -> term263 Nt=5 TF=5 maxTF=1 @{0 86 2}
16th -> term281 Nt=5 TF=5 maxTF=1 @{0 94 6}
17 -> term203 Nt=4 TF=4 maxTF=1 @{0 103 6}
17th -> term328 Nt=5 TF=5 maxTF=1 @{0 111 2}
18 -> term216 Nt=4 TF=4 maxTF=1 @{0 120 0}
185 -> term809 Nt=2 TF=2 maxTF=1 @{0 127 4}
18th -> term218 Nt=6 TF=6 maxTF=1 @{0 130 2}
19 -> term385 Nt=5 TF=5 maxTF=1 @{0 140 6}
1955 -> te

In [65]:
term = "ai"
index.getLexicon()[term].toString()

'term73 Nt=62 TF=62 maxTF=1 @{0 767 0}'

In [45]:
#how often do the terms occur
term_freq_dict = {}

for kv in index.getLexicon():
    term_freq_dict[kv.getKey()] = kv.getValue().frequency

In [54]:
for kv in index.getLexicon():
    print(kv.getValue(), kv.getValue().frequency)

term821 Nt=1 TF=1 maxTF=1 @{0 0 0} 1
term588 Nt=2 TF=2 maxTF=1 @{0 2 2} 2
term327 Nt=2 TF=2 maxTF=1 @{0 5 6} 2
term542 Nt=3 TF=3 maxTF=1 @{0 9 6} 3
term580 Nt=2 TF=2 maxTF=1 @{0 15 4} 2
term118 Nt=3 TF=3 maxTF=1 @{0 19 6} 3
term1135 Nt=4 TF=4 maxTF=1 @{0 25 4} 4
term381 Nt=2 TF=2 maxTF=1 @{0 32 4} 2
term1090 Nt=4 TF=4 maxTF=1 @{0 36 0} 4
term117 Nt=5 TF=5 maxTF=1 @{0 43 2} 5
term1206 Nt=3 TF=3 maxTF=1 @{0 52 4} 3
term525 Nt=4 TF=4 maxTF=1 @{0 57 6} 4
term1045 Nt=4 TF=4 maxTF=1 @{0 65 0} 4
term204 Nt=4 TF=4 maxTF=1 @{0 73 2} 4
term1055 Nt=4 TF=4 maxTF=1 @{0 78 4} 4
term263 Nt=5 TF=5 maxTF=1 @{0 86 2} 5
term281 Nt=5 TF=5 maxTF=1 @{0 94 6} 5
term203 Nt=4 TF=4 maxTF=1 @{0 103 6} 4
term328 Nt=5 TF=5 maxTF=1 @{0 111 2} 5
term216 Nt=4 TF=4 maxTF=1 @{0 120 0} 4
term809 Nt=2 TF=2 maxTF=1 @{0 127 4} 2
term218 Nt=6 TF=6 maxTF=1 @{0 130 2} 6
term385 Nt=5 TF=5 maxTF=1 @{0 140 6} 5
term1011 Nt=1 TF=1 maxTF=1 @{0 149 6} 1
term807 Nt=2 TF=2 maxTF=1 @{0 152 2} 2
term806 Nt=2 TF=2 maxTF=1 @{0 155 0} 2
t

In [49]:
for key, value in term_freq_dict.items():
    print(key, value)

0 1
02 2
1 2
10 3
10th 2
11 3
11th 4
12 2
12th 4
13 5
13th 3
14 4
14th 4
15 4
15th 4
16 5
16th 5
17 4
17th 5
18 4
185 2
18th 6
19 5
1955 1
1970 2
1971 2
1985 1
1986 1
1987 1
1988 1
1989 1
1991 1
1992 1
1994 1
1995 1
1996 1
1997 2
1998 2
1999 2
19th 6
2 4
20 5
2000 1
2001 4
2002 6
2003 7
2004 6
2005 10
2006 10
2007 5
2008 8
2009 2
2010 4
2011 5
2012 11
2013 4
2014 5
2015 3
2016 6
2017 1
2018 2
2019 2
2020 1
2027 1
20th 5
21 6
21st 4
22 7
22nd 3
23 5
23rd 1
24 10
24th 1
25 7
25th 2
26 6
26th 1
27 6
27th 2
28 5
28th 1
29 3
29th 1
2nd 8
3 7
30 3
30th 3
31 6
31st 1
32nd 1
33rd 1
34th 1
35th 1
36th 1
37th 1
38th 1
39th 1
3d 1
3rd 1
4 11
4099 1
5 5
50 1
50th 1
6 5
60th 2
6g 2
7 3
7th 2
8 3
9 4
900mhz 1
92 1
94 1
95 1
96 1
97 2
98 2
99 2
9th 1
aaai 2
aachen 1
abuja 2
acceler 1
accept 1
accord 1
accur 1
achiev 4
act 2
action 1
actor 1
acut 1
adapt 8
adelaid 1
adjust 1
adopt 1
advanc 44
advantag 1
advent 2
adversari 1
aerial 1
aerospac 2
aesthet 2
affect 2
ag 6
agent 15
agricultur 1
ai 62
aid 1


In [52]:
#sort all terms from the index in descending order 
term_freq_dict_sorted = sorted(term_freq_dict.items(), key=lambda x: x[1], reverse=True)

In [55]:
#show the 30 nost popular entries
top_k = 30
term_freq_dict_sorted[:top_k]

[('intellig', 992),
 ('artifici', 958),
 ('confer', 95),
 ('proceed', 93),
 ('system', 63),
 ('ai', 62),
 ('approach', 50),
 ('us', 45),
 ('advanc', 44),
 ('applic', 42),
 ('comput', 39),
 ('base', 37),
 ('gener', 36),
 ('learn', 35),
 ('human', 31),
 ('ki', 30),
 ('nation', 29),
 ('research', 29),
 ('modern', 27),
 ('septemb', 27),
 ('german', 25),
 ('model', 25),
 ('educ', 24),
 ('network', 23),
 ('distribut', 22),
 ('intern', 22),
 ('annual', 21),
 ('develop', 21),
 ('germani', 21),
 ('impact', 21)]

## Search engine

In [56]:
#define our search engine
search_engine = pt.BatchRetrieve(index, wmodel="Tf")

In [149]:
search_engine.search("artificial")

,qid,docid,docno,rank,score,query
0,1,386,https://www.bibsonomy.org/bibtex/2d8dbc9f46979...,0,3.0,artificial
1,1,591,https://www.bibsonomy.org/bibtex/2251f83f9d1c9...,1,3.0,artificial
2,1,42,https://www.bibsonomy.org/bibtex/284b5b3f38e04...,2,2.0,artificial
3,1,45,https://www.bibsonomy.org/bibtex/274ab987ae955...,3,2.0,artificial
4,1,55,https://www.bibsonomy.org/bibtex/2dc0153e9b5a9...,4,2.0,artificial
...,...,...,...,...,...,...
892,1,987,https://www.bibsonomy.org/bibtex/2f4c4085c258c...,892,1.0,artificial
893,1,988,https://www.bibsonomy.org/bibtex/228aef007987a...,893,1.0,artificial
894,1,995,https://www.bibsonomy.org/bibtex/2b44f329ab5e2...,894,1.0,artificial
895,1,996,https://www.bibsonomy.org/bibtex/2451eb56bc876...,895,1.0,artificial


So the `search()` method returns a dataframe with columns:
 - `qid`: this is by default "1", since it's our first and only query
 - `docid`: Terrier' internal integer for each document
 - `docno`: the external (string) unique identifier for each document
 - `score`: since we use the `Tf` weighting model, this score corresponds the total frequency of the query (terms) in each document
 - `rank`: A handy attribute showing the descending order by score
 - `query`: the input query

As expected, the `Tf` weighting model used here only counts the frequencies of the query terms in each document, i.e.:
$$
score(d,q) = \sum_{t \in q} tf_{t,d}
$$

In [59]:
search_engine.search("intelligent education")

,qid,docid,docno,rank,score,query
0,1,731,https://www.bibsonomy.org/bibtex/2b476e9f37ca1...,0,5.0,intelligent education
1,1,631,https://www.bibsonomy.org/bibtex/226a944b1f244...,1,3.0,intelligent education
2,1,708,https://www.bibsonomy.org/bibtex/224ac67b53792...,2,3.0,intelligent education
3,1,788,https://www.bibsonomy.org/bibtex/21c47c79fb260...,3,3.0,intelligent education
4,1,989,https://www.bibsonomy.org/bibtex/2dfb01a4c7c8d...,4,3.0,intelligent education
...,...,...,...,...,...,...
900,1,984,https://www.bibsonomy.org/bibtex/2025ecfe8993f...,900,1.0,intelligent education
901,1,985,https://www.bibsonomy.org/bibtex/24bc8dd6afe36...,901,1.0,intelligent education
902,1,987,https://www.bibsonomy.org/bibtex/2f4c4085c258c...,902,1.0,intelligent education
903,1,995,https://www.bibsonomy.org/bibtex/2b44f329ab5e2...,903,1.0,intelligent education


In [91]:
search_engine.search("intelligent education")

,qid,docid,docno,rank,score,query
0,1,731,https://www.bibsonomy.org/bibtex/2b476e9f37ca1...,0,5.0,intelligent education
1,1,631,https://www.bibsonomy.org/bibtex/226a944b1f244...,1,3.0,intelligent education
2,1,708,https://www.bibsonomy.org/bibtex/224ac67b53792...,2,3.0,intelligent education
3,1,788,https://www.bibsonomy.org/bibtex/21c47c79fb260...,3,3.0,intelligent education
4,1,989,https://www.bibsonomy.org/bibtex/2dfb01a4c7c8d...,4,3.0,intelligent education
...,...,...,...,...,...,...
900,1,984,https://www.bibsonomy.org/bibtex/2025ecfe8993f...,900,1.0,intelligent education
901,1,985,https://www.bibsonomy.org/bibtex/24bc8dd6afe36...,901,1.0,intelligent education
902,1,987,https://www.bibsonomy.org/bibtex/2f4c4085c258c...,902,1.0,intelligent education
903,1,995,https://www.bibsonomy.org/bibtex/2b44f329ab5e2...,903,1.0,intelligent education


In [ ]:
#what do we need for the inverse document frequency?

In [62]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [150]:
#stem the term its base form
stem = stemmer.stem("artificial")
stem

'artifici'

In [64]:
#in how many documents does the stem 'learn' occur?
lexicon = index.getLexicon()
lexicon[stem].getDocumentFrequency()

33

In [66]:
#how many documents occur in our index?
index.getCollectionStatistics().numberOfDocuments

1000

In [67]:
def get_idf_for_term(term, index):
  lex = index.getLexicon()
  stemmed_term = stemmer.stem(term)

  if not stemmed_term in lex:
    return 
    
  lex_entry = lex[stemmed_term]

  df_term = lex_entry.getDocumentFrequency()
  N = index.getCollectionStatistics().numberOfDocuments
  
  #inverse document frequency
  idf = N/df_term

  #apply logarithm(base 10) to idf
  log_idf = np.emath.logn(10, idf)

  return log_idf

In [ ]:
get_idf_for_term("learning", index)

In [156]:
#define your own tf_idf method here:    
def calc_tf_idf(query, docno, index):
    #remember that tfidf is the product of two components
    #hint: the tf model search result contains tf frequencies
    terms = query.split()
    tf_sum, idf_sum = 0, 0
    
    for term in terms: 
        # Get idf
        idf_sum += get_idf_for_term(term, index)
        
        # Get tf 
        df_result = search_engine.search(query)
        tf = df_result.query('docno == @docno')['score'].iloc[0]
        tf_sum += 1 + np.emath.logn(10, tf)
    return tf_sum * idf_sum

In [157]:
query = "intelligent education"
docno = "https://www.bibsonomy.org/bibtex/201f2eb94f27fe662c37249be37619d8b/dblp"

print(f"The tf-idf for query: {query} and document: {docno} is {calc_tf_idf(query, docno, index):.2f}")

The tf-idf for query: intelligent education and document: https://www.bibsonomy.org/bibtex/201f2eb94f27fe662c37249be37619d8b/dblp is 4.38


In [158]:
query = "Artificial Intelligence"
docno = "https://www.bibsonomy.org/bibtex/201f2eb94f27fe662c37249be37619d8b/dblp"

print(f"The tf-idf for query: {query} and document: {docno} is {calc_tf_idf(query, docno, index):.2f}")

The tf-idf for query: Artificial Intelligence and document: https://www.bibsonomy.org/bibtex/201f2eb94f27fe662c37249be37619d8b/dblp is 0.24


### Search mit `tf_idf` model

In [154]:
search_engine_tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF")

In [155]:
search_engine_tfidf.search("intelligent education")

,qid,docid,docno,rank,score,query
0,1,276,https://www.bibsonomy.org/bibtex/201f2eb94f27f...,0,4.555365,intelligent education
1,1,776,https://www.bibsonomy.org/bibtex/2763f67bf4e7d...,1,4.555365,intelligent education
2,1,833,https://www.bibsonomy.org/bibtex/29cc0434b3e50...,2,4.555365,intelligent education
3,1,20,https://www.bibsonomy.org/bibtex/259133e526641...,3,4.209571,intelligent education
4,1,58,https://www.bibsonomy.org/bibtex/259133e526641...,4,4.209571,intelligent education
...,...,...,...,...,...,...
900,1,200,https://www.bibsonomy.org/bibtex/22113c27f2328...,900,0.322746,intelligent education
901,1,203,https://www.bibsonomy.org/bibtex/23934d5315fab...,901,0.322746,intelligent education
902,1,238,https://www.bibsonomy.org/bibtex/2201d72f3a55d...,902,0.322746,intelligent education
903,1,279,https://www.bibsonomy.org/bibtex/268ebe1d6569e...,903,0.322746,intelligent education


In [ ]:
#Now we want to build an index with multiple fields

In [159]:
df_ai_filtered.columns

Index(['id', 'tags', 'label', 'description', 'date', 'changeDate', 'url',
       'pub-type', 'year', 'author', 'authors', 'publisher', 'docno', 'text'],
      dtype='object')

In [160]:
#transfrom dataframe into list of dictionaries
ai_dict = df_ai_filtered.to_dict(orient='records')

In [162]:
ai_dict[0]

{'id': 'https://www.bibsonomy.org/bibtex/24e338b04bc2abafd9fa0a1f4fe79103a/ijtsrd',
 'tags': ['710-714'],
 'label': 'Artificial Intelligence in Agriculture',
 'description': '',
 'date': '2021-04-13 13:15:04',
 'changeDate': '2021-04-13 13:15:04',
 'url': 'https://www.ijtsrd.com/engineering/electrical-engineering/38513/artificial-intelligence-in-agriculture/matthew-n-o-sadiku',
 'pub-type': 'article',
 'year': '2021',
 'author': ['Matthew N. O. Sadiku | Sarhan M. Musa | Abayomi Ajayi-Majebi'],
 'authors': [{'first': 'Matthew N. O. Sadiku | Sarhan M. Musa | Abayomi',
   'last': 'Ajayi-Majebi'}],
 'publisher': nan,
 'docno': 'https://www.bibsonomy.org/bibtex/24e338b04bc2abafd9fa0a1f4fe79103a/ijtsrd',
 'text': 'Artificial Intelligence in Agriculture'}

## Create `pt.IterDictIndexer`

In [163]:
index_folder_mult = "./ai_index_mult"

fields=['docno', 'text', 'tags', 'description']

indexer_mult = pt.IterDictIndexer(index_folder_mult, meta={'docno': 200, 'text': 4096}, overwrite=True)
index_ref_mult = indexer_mult.index(ai_dict, fields=fields)

print(f"path to our index: {index_ref_mult.toString()}")

path to our index: ./ai_index_mult/data.properties


In [164]:
index_mult = pt.IndexFactory.of(index_ref_mult)

In [166]:
print(index_mult.getCollectionStatistics().toString())

Number of documents: 1000
Number of terms: 2180
Number of postings: 14484
Number of fields: 4
Number of tokens: 16409
Field names: [docno, text, tags, description]
Positions:   false



In [167]:
search_engine_mult = pt.BatchRetrieve(index_mult, wmodel="TF_IDF")

In [168]:
res = search_engine_mult.search("intelligence")
res

,qid,docid,docno,rank,score,query
0,1,708,https://www.bibsonomy.org/bibtex/224ac67b53792...,0,0.935614,intelligence
1,1,996,https://www.bibsonomy.org/bibtex/2451eb56bc876...,1,0.922999,intelligence
2,1,335,https://www.bibsonomy.org/bibtex/29a73b5ea0550...,2,0.878748,intelligence
3,1,502,https://www.bibsonomy.org/bibtex/2f7cf941158e2...,3,0.878748,intelligence
4,1,653,https://www.bibsonomy.org/bibtex/2cd2e7a05ba63...,4,0.878748,intelligence
...,...,...,...,...,...,...
938,1,318,https://www.bibsonomy.org/bibtex/2b5cd3e615563...,938,0.388718,intelligence
939,1,610,https://www.bibsonomy.org/bibtex/2b4e425ae3b44...,939,0.388718,intelligence
940,1,342,https://www.bibsonomy.org/bibtex/210b214d3428a...,940,0.375911,intelligence
941,1,352,https://www.bibsonomy.org/bibtex/299846faf0a0d...,941,0.369818,intelligence


In [170]:
#save our data
pickle.dump(df_ai_filtered, open("./ai_publications.pkl", "wb"))